# Face Super-Resolution with UNet
## Goal: Enhance 64x64 face images to 256x256 using UNet

Based on: https://www.kaggle.com/code/ashishjangra27/face-resolution-enhancement-with-unet


## Step 1: Install Dependencies


In [ ]:
# Install required packages
%pip install -q numpy pillow matplotlib opencv-python scikit-image tensorflow==2.19.0


## Step 2: Import Libraries


In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from PIL import Image
import matplotlib.pyplot as plt
import cv2
from skimage import io, transform

# Check GPU availability
print(f"TensorFlow version: {tf.__version__}")
print(f"GPU available: {tf.config.list_physical_devices('GPU')}")

# Set random seeds for reproducibility
tf.random.set_seed(42)
np.random.seed(42)


## Step 3: Define UNet Architecture


In [ ]:
def conv_block(x, filters, kernel_size=3, strides=1, padding='same'):
    """Convolutional block with BatchNorm and ReLU"""
    x = layers.Conv2D(filters, kernel_size, strides=strides, padding=padding)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    return x

def upsample_block(x, filters, kernel_size=3, strides=2, padding='same'):
    """Upsampling block with Conv2DTranspose"""
    x = layers.Conv2DTranspose(filters, kernel_size, strides=strides, padding=padding)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    return x

def build_unet(input_shape=(64, 64, 3)):
    """Build UNet model for face super-resolution (64x64 -> 256x256)"""
    inputs = keras.Input(shape=input_shape)

    # Encoder (Downsampling)
    # Block 1: 64x64 -> 32x32
    conv1 = conv_block(inputs, 64)
    conv1 = conv_block(conv1, 64)
    pool1 = layers.MaxPooling2D(pool_size=(2, 2))(conv1)

    # Block 2: 32x32 -> 16x16
    conv2 = conv_block(pool1, 128)
    conv2 = conv_block(conv2, 128)
    pool2 = layers.MaxPooling2D(pool_size=(2, 2))(conv2)

    # Block 3: 16x16 -> 8x8
    conv3 = conv_block(pool2, 256)
    conv3 = conv_block(conv3, 256)
    pool3 = layers.MaxPooling2D(pool_size=(2, 2))(conv3)

    # Block 4: 8x8 -> 4x4
    conv4 = conv_block(pool3, 512)
    conv4 = conv_block(conv4, 512)
    pool4 = layers.MaxPooling2D(pool_size=(2, 2))(conv4)

    # Bottleneck: 4x4
    conv5 = conv_block(pool4, 1024)
    conv5 = conv_block(conv5, 1024)

    # Decoder (Upsampling)
    # Block 6: 4x4 -> 8x8
    up6 = upsample_block(conv5, 512)
    concat6 = layers.Concatenate()([conv4, up6])
    conv6 = conv_block(concat6, 512)
    conv6 = conv_block(conv6, 512)

    # Block 7: 8x8 -> 16x16
    up7 = upsample_block(conv6, 256)
    concat7 = layers.Concatenate()([conv3, up7])
    conv7 = conv_block(concat7, 256)
    conv7 = conv_block(conv7, 256)

    # Block 8: 16x16 -> 32x32
    up8 = upsample_block(conv7, 128)
    concat8 = layers.Concatenate()([conv2, up8])
    conv8 = conv_block(concat8, 128)
    conv8 = conv_block(conv8, 128)

    # Block 9: 32x32 -> 64x64
    up9 = upsample_block(conv8, 64)
    concat9 = layers.Concatenate()([conv1, up9])
    conv9 = conv_block(concat9, 64)
    conv9 = conv_block(conv9, 64)

    # Final upsampling: 64x64 -> 256x256 (4x upsampling)
    up10 = upsample_block(conv9, 64, strides=4)

    # Output layer
    outputs = layers.Conv2D(3, 1, activation='sigmoid', padding='same')(up10)

    model = keras.Model(inputs, outputs, name='FaceSuperResolutionUNet')
    return model

# Build the model
model = build_unet()
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='mse',
    metrics=['mae']
)

print("Model Summary:")
model.summary()


## Step 4: Image Preprocessing Functions


In [ ]:
def preprocess_image(image_path, target_size=(64, 64)):
    """Load and preprocess image for model input"""
    # Load image
    img = Image.open(image_path).convert('RGB')

    # Resize to target size
    img_resized = img.resize(target_size, Image.LANCZOS)

    # Convert to numpy array and normalize
    img_array = np.array(img_resized, dtype=np.float32) / 255.0

    # Add batch dimension
    img_batch = np.expand_dims(img_array, axis=0)

    return img_batch, img_resized

def postprocess_image(prediction):
    """Convert model prediction back to image"""
    # Remove batch dimension
    img_array = prediction[0]

    # Denormalize
    img_array = np.clip(img_array * 255.0, 0, 255).astype(np.uint8)

    # Convert to PIL Image
    img = Image.fromarray(img_array)

    return img

def create_sample_face_image():
    """Create a simple synthetic face image for testing"""
    # Create a 64x64 synthetic face-like image
    img = np.zeros((64, 64, 3), dtype=np.uint8)

    # Face outline (oval)
    cv2.ellipse(img, (32, 35), (25, 30), 0, 0, 360, (255, 220, 177), -1)

    # Eyes
    cv2.circle(img, (25, 25), 3, (0, 0, 0), -1)
    cv2.circle(img, (39, 25), 3, (0, 0, 0), -1)

    # Nose
    cv2.ellipse(img, (32, 35), (2, 4), 0, 0, 360, (200, 180, 150), -1)

    # Mouth
    cv2.ellipse(img, (32, 45), (6, 3), 0, 0, 180, (200, 100, 100), -1)

    return Image.fromarray(img)

print("Preprocessing functions defined successfully!")


## Step 5: Demo with Sample Image


In [ ]:
# Create a sample face image
sample_face = create_sample_face_image()

# Save the sample image
sample_face.save('sample_face_64x64.png')

# Preprocess the image
input_batch, input_img = preprocess_image('sample_face_64x64.png')

print(f"Input image shape: {input_batch.shape}")
print(f"Input image size: {input_img.size}")


## Step 6: Apply UNet Model


In [ ]:
# Make prediction
print("Making prediction...")
prediction = model.predict(input_batch, verbose=1)

print(f"Prediction shape: {prediction.shape}")
print(f"Expected output size: 256x256 pixels")


## Step 7: Postprocess and Display Results


In [ ]:
# Convert prediction to image
enhanced_img = postprocess_image(prediction)

# Save enhanced image
enhanced_img.save('enhanced_face_256x256.png')

# Display comparison
fig, axes = plt.subplots(1, 2, figsize=(12, 6))

# Original 64x64 image
axes[0].imshow(input_img)
axes[0].set_title('Original Face (64x64)', fontsize=14)
axes[0].axis('off')

# Enhanced 256x256 image
axes[1].imshow(enhanced_img)
axes[1].set_title('Enhanced Face (256x256)', fontsize=14)
axes[1].axis('off')

plt.tight_layout()
plt.show()

print(f"Original image size: {input_img.size}")
print(f"Enhanced image size: {enhanced_img.size}")
print("Images saved as 'sample_face_64x64.png' and 'enhanced_face_256x256.png'")


## Step 8: Test with Your Own Image (Optional)


In [ ]:
# Uncomment and modify this section to test with your own image
#
# # Replace 'your_image.jpg' with the path to your image
# your_image_path = 'your_image.jpg'
#
# # Preprocess your image
# input_batch, input_img = preprocess_image(your_image_path)
#
# # Make prediction
# prediction = model.predict(input_batch, verbose=1)
#
# # Convert to image
# enhanced_img = postprocess_image(prediction)
#
# # Display results
# fig, axes = plt.subplots(1, 2, figsize=(12, 6))
# axes[0].imshow(input_img)
# axes[0].set_title('Your Original Image (64x64)')
# axes[0].axis('off')
#
# axes[1].imshow(enhanced_img)
# axes[1].set_title('Enhanced Image (256x256)')
# axes[1].axis('off')
#
# plt.tight_layout()
# plt.show()

print("To test with your own image, uncomment and modify the code above.")


## Summary

This notebook demonstrates:
1. ✅ **UNet Architecture**: Custom UNet for 64x64 → 256x256 face enhancement
2. ✅ **Image Preprocessing**: Loading and resizing images to 64x64
3. ✅ **Model Application**: Running inference with the UNet model
4. ✅ **Output Handling**: Converting predictions to 256x256 images
5. ✅ **Visualization**: Side-by-side comparison of original and enhanced images

**Next Steps for Production:**
- Train the model on a face dataset (e.g., CelebA)
- Use pre-trained weights from the Kaggle notebook
- Implement data augmentation for better generalization
- Add perceptual loss for better visual quality
